## Import modules and define functions

In [1]:
# Import modules
import pandas as pd
import numpy as np
import os
import warnings
from category_encoders.target_encoder import TargetEncoder
warnings.filterwarnings("ignore")

#0.25 spring, 0.75 autumn.
def date_to_year(df):
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df["month"] = pd.to_datetime(df["date"]).dt.month

    df["year"] = np.where(df["month"] <= 6, df["year"] + 0.25, df["year"] + 0.75)
    return df

def valuation_date_to_year(df):
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df["month"] = pd.to_datetime(df["date"]).dt.month

    conditions = [
    (df['month'] > 2) & (df['month'] < 9),
    (df['month'] >= 9),
    (df['month'] < 3)
    ]

    choices = ['Spring', 'Fall', 'Fall']
    df['semester'] = np.select(conditions, choices)
    df.loc[(df['month'] < 3), 'year'] -= 1
    df["year"] = np.where(df["semester"] == "Spring", df["year"] + 0.25, df["year"] + 0.75)
    
    return df

def calculate_age(row):
    date_of_birth = pd.to_datetime(row['date_of_birth'])
    date = pd.to_datetime(row['date'])
    age = date.year - date_of_birth.year - ((date.month, date.day) < (date.month, date.day))
    return age

print("Packages installed")

Packages installed


## Read CSV

In [2]:
dataframes=[]
for dirname, _, filenames in os.walk('RawData/'):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirname,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported') #Tar ca. 10 sekunder å lese alle filene

appearances_df (1533437, 13)
clubs_df (426, 17)
club_games_df (133132, 11)
competitions_df (43, 10)
games_df (66566, 23)
game_events_df (687761, 10)
game_lineups_df (2319806, 9)
players_df (30482, 23)
player_valuations_df (465228, 5)
Data imported


## 1. Games & Appearances

In [3]:
games_df = games_df[["game_id", "date", "home_club_id", "away_club_id", "home_club_goals", "away_club_goals"]]
appearances_df = appearances_df[["game_id", "player_id", "player_club_id", "yellow_cards", "red_cards", "goals", "assists", "minutes_played"]]

games_df = pd.merge(games_df, appearances_df, on="game_id")

games_df['Club Goals'] = games_df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] else row['away_club_goals'], axis=1)
games_df['Club Goals Conceded'] = games_df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] else row['home_club_goals'], axis=1)
games_df = date_to_year(games_df)
games_df.head()

,game_id,date,home_club_id,away_club_id,home_club_goals,away_club_goals,player_id,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,Club Goals,Club Goals Conceded,year,month
0,2321044,2013-08-18,16,23,2.0,1.0,121404,23,0,0,0,0,90,1.0,2.0,2013.75,8
1,2321044,2013-08-18,16,23,2.0,1.0,125103,16,0,0,0,0,1,2.0,1.0,2013.75,8
2,2321044,2013-08-18,16,23,2.0,1.0,1659,23,0,0,0,0,77,1.0,2.0,2013.75,8
3,2321044,2013-08-18,16,23,2.0,1.0,16831,23,0,0,0,0,90,1.0,2.0,2013.75,8
4,2321044,2013-08-18,16,23,2.0,1.0,26,16,0,0,0,0,90,2.0,1.0,2013.75,8


## 2. Player performance

In [4]:
games_df = games_df[["player_id", "player_club_id", "yellow_cards", "red_cards", "goals", "assists", "minutes_played", "Club Goals", "Club Goals Conceded", "year"]]
player_performance_df = games_df.groupby(['player_id', 'player_club_id', 'year']).agg({
    'Club Goals': 'sum',
    'Club Goals Conceded': 'sum',
    'goals': 'sum', 
    'assists': 'sum', 
    'red_cards': 'sum', 
    'yellow_cards': 'sum', 
    'minutes_played': 'sum'
}).reset_index()

player_performance_df.head()

,player_id,player_club_id,year,Club Goals,Club Goals Conceded,goals,assists,red_cards,yellow_cards,minutes_played
0,10,398,2012.75,31.0,14.0,11,1,0,6,1483
1,10,398,2013.25,24.0,15.0,5,2,0,2,1102
2,10,398,2013.75,14.0,21.0,4,2,0,1,950
3,10,398,2014.25,24.0,22.0,4,3,0,1,1270
4,10,398,2014.75,31.0,18.0,4,3,0,2,496


## 

## 3. Player valuation

In [5]:
player_valuations_df = valuation_date_to_year(player_valuations_df)[["player_id", "market_value_in_eur", "year", "date"]]
players_performance_value_df = pd.merge(player_performance_df, player_valuations_df, on=["player_id", "year"])


players_performance_value_df.sort_values(by=['player_id', 'year', 'date'], ascending=[True, True, False], inplace=True)

# Fjern duplikater basert på 'player_id' og 'year', behold raden med nyeste dato
players_performance_value_df.drop_duplicates(subset=['player_id', 'year'], keep='first', inplace=True)
players_performance_value_df.head()

,player_id,player_club_id,year,Club Goals,Club Goals Conceded,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date
0,10,398,2012.75,31.0,14.0,11,1,0,6,1483,4000000,2013-01-14
1,10,398,2013.25,24.0,15.0,5,2,0,2,1102,2000000,2013-06-19
2,10,398,2013.75,14.0,21.0,4,2,0,1,950,1000000,2014-01-07
3,10,398,2014.25,24.0,22.0,4,3,0,1,1270,1000000,2014-07-07
4,10,398,2014.75,31.0,18.0,4,3,0,2,496,1000000,2015-01-07


## 4. Player Characteristic

In [6]:
players_characteristics_df = players_df[['player_id', 'name', 'country_of_birth', 'country_of_citizenship','date_of_birth', 'height_in_cm', 'sub_position']]
players_characteristics_df['country_of_birth'].fillna(players_characteristics_df['country_of_citizenship'], inplace=True)
players_characteristics_df.dropna(inplace=True)
players_characteristics_df.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward
1,26,Roman Weidenfeller,Germany,Germany,1980-08-06,190.0,Goalkeeper
4,80,Tom Starke,East Germany (GDR),Germany,1981-03-18,194.0,Goalkeeper
7,132,Tomas Rosicky,CSSR,Czech Republic,1980-10-04,179.0,Attacking Midfield
8,162,Marc Ziegler,Germany,Germany,1976-06-13,193.0,Goalkeeper


## 5. Combine and Calculate Age

In [7]:
total_df = pd.merge(players_characteristics_df, players_performance_value_df, on='player_id', how='left')
total_df.dropna(inplace=True)

total_df['Age'] = total_df.apply(calculate_age, axis=1)
total_df.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,player_club_id,year,Club Goals,Club Goals Conceded,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,Age
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,4000000.0,2013-01-14,35
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,2000000.0,2013-06-19,35
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,1000000.0,2014-01-07,36
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,1000000.0,2014-07-07,36
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,1000000.0,2015-01-07,37


## 6. Add league as feature

In [8]:
#df_player = pd.read_csv('/work/cleaned_data_28.csv', sep=",", encoding="UTF-8")
df_player = total_df
df_player = df_player.rename(columns={'player_club_id': 'club_id'})
clubs_df = clubs_df[["club_id", "domestic_competition_id"]]

df = pd.merge(df_player, clubs_df, on='club_id', how="left")
df.head()


,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,Club Goals,Club Goals Conceded,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,Age,domestic_competition_id
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,4000000.0,2013-01-14,35,IT1
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,2000000.0,2013-06-19,35,IT1
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,1000000.0,2014-01-07,36,IT1
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,1000000.0,2014-07-07,36,IT1
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,1000000.0,2015-01-07,37,IT1


## Normalize market value

In [9]:
df['Logarithmic Market Value'] = np.log10(df['market_value_in_eur'])
df.head()


,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,Club Goals,...,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,Age,domestic_competition_id,Logarithmic Market Value
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,...,11.0,1.0,0.0,6.0,1483.0,4000000.0,2013-01-14,35,IT1,6.60206
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,...,5.0,2.0,0.0,2.0,1102.0,2000000.0,2013-06-19,35,IT1,6.30103
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,...,4.0,2.0,0.0,1.0,950.0,1000000.0,2014-01-07,36,IT1,6.00000
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,...,4.0,3.0,0.0,1.0,1270.0,1000000.0,2014-07-07,36,IT1,6.00000
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,...,4.0,3.0,0.0,2.0,496.0,1000000.0,2015-01-07,37,IT1,6.00000


### Add Goals/90, Assists/90 and Goals Difference

In [10]:
df["Goals/Game"] = df["goals"]*90/df["minutes_played"]
df["Assists/Game"] = df["assists"]*90/df["minutes_played"]
df["Goal Difference"] = df["Club Goals"]-df["Club Goals Conceded"]
df.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,Club Goals,...,yellow_cards,minutes_played,market_value_in_eur,date,Age,domestic_competition_id,Logarithmic Market Value,Goals/Game,Assists/Game,Goal Difference
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,...,6.0,1483.0,4000000.0,2013-01-14,35,IT1,6.60206,0.667566,0.060688,17.0
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,...,2.0,1102.0,2000000.0,2013-06-19,35,IT1,6.30103,0.408348,0.163339,9.0
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,...,1.0,950.0,1000000.0,2014-01-07,36,IT1,6.00000,0.378947,0.189474,-7.0
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,...,1.0,1270.0,1000000.0,2014-07-07,36,IT1,6.00000,0.283465,0.212598,2.0
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,...,2.0,496.0,1000000.0,2015-01-07,37,IT1,6.00000,0.725806,0.544355,13.0


# Target Encode

In [11]:
# Initialize TargetEncoder
encoder = TargetEncoder()

# Specify columns to target encode
columns_to_encode = ['country_of_citizenship', 'club_id','sub_position', 'domestic_competition_id']
df['club_id'] = df['club_id'].astype(str)
# Fit and transform the DataFrame with target encoding
df_encoded = encoder.fit_transform(df[columns_to_encode], df['market_value_in_eur'])

# Merge the encoded DataFrame with the original DataFrame
df = pd.concat([df, df_encoded.add_suffix('_encoded')], axis=1)
# Display the resulting DataFrame
df.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,Club Goals,...,Age,domestic_competition_id,Logarithmic Market Value,Goals/Game,Assists/Game,Goal Difference,country_of_citizenship_encoded,club_id_encoded,sub_position_encoded,domestic_competition_id_encoded
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,...,35,IT1,6.60206,0.667566,0.060688,17.0,7.191504e+06,7.836111e+06,5.331868e+06,7.182738e+06
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,...,35,IT1,6.30103,0.408348,0.163339,9.0,7.191504e+06,7.836111e+06,5.331868e+06,7.182738e+06
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,...,36,IT1,6.00000,0.378947,0.189474,-7.0,7.191504e+06,7.836111e+06,5.331868e+06,7.182738e+06
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,...,36,IT1,6.00000,0.283465,0.212598,2.0,7.191504e+06,7.836111e+06,5.331868e+06,7.182738e+06
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,...,37,IT1,6.00000,0.725806,0.544355,13.0,7.191504e+06,7.836111e+06,5.331868e+06,7.182738e+06


In [12]:

# Define a dictionary to map the original positions to the desired abbreviations
position_mapping = {
    'Centre-Forward': 'CF',
    'Goalkeeper': 'GK',
    'Attacking Midfield': 'AM',
    'Central Midfield': 'CM',
    'Defensive Midfield': 'DM',
    'Left-Back': 'LB',
    'Centre-Back': 'CB',
    'Right-Back': 'RB',
    'Second Striker': 'SS',
    'Right Winger': 'RW',
    'Right Midfield': 'RM',
    'Left Winger': 'LW',
    'Left Midfield': 'LM'
}


# Replace the positions with the abbreviations
df['sub_position'] = df['sub_position'].replace(position_mapping)

In [13]:

# Perform one-hot encoding
one_hot_encoded = pd.get_dummies(df['sub_position'], prefix='POS')

# Concatenate the one-hot encoded columns with the original DataFrame
df_encoded = pd.concat([df, one_hot_encoded], axis=1)
cols_to_convert = ["POS_AM", "POS_CB", "POS_CF", "POS_CM", "POS_DM", "POS_GK", "POS_LB", "POS_LM", "POS_LW", "POS_RB","POS_RM", "POS_RW", "POS_SS"]
df_encoded[cols_to_convert] = df_encoded[cols_to_convert].astype(int)
df_encoded.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,Club Goals,...,POS_CM,POS_DM,POS_GK,POS_LB,POS_LM,POS_LW,POS_RB,POS_RM,POS_RW,POS_SS
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,CF,398.0,2012.75,31.0,...,0,0,0,0,0,0,0,0,0,0
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,CF,398.0,2013.25,24.0,...,0,0,0,0,0,0,0,0,0,0
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,CF,398.0,2013.75,14.0,...,0,0,0,0,0,0,0,0,0,0
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,CF,398.0,2014.25,24.0,...,0,0,0,0,0,0,0,0,0,0
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,CF,398.0,2014.75,31.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df = df_encoded
df["Club Value"] = np.log10(df['club_id_encoded'])
df["Country Value"] = np.log10(df['country_of_citizenship_encoded'])
df["League Value"] = np.log10(df['domestic_competition_id_encoded'])
df["Position Value"] = np.log10(df['sub_position_encoded'])


df = df.drop(["country_of_citizenship", "domestic_competition_id", "club_id", "player_id", "date"], axis=1)
df = df.drop(["country_of_citizenship_encoded", "club_id_encoded", "sub_position_encoded", "domestic_competition_id_encoded", "country_of_birth", "date_of_birth", "sub_position"], axis=1)

df.rename(columns={"height_in_cm" : "Height", "year":"Year", "goals":"Goals", "name" : "Name"}, inplace=True)
df.rename(columns={"assists" : "Assists", "red_cards":"Red Cards", "yellow_cards":"Yellow Cards", "minutes_played":"Minutes Played", "market_value_in_eur":"Market Value"}, inplace=True)

df.head()

,Name,Height,Year,Club Goals,Club Goals Conceded,Goals,Assists,Red Cards,Yellow Cards,Minutes Played,...,POS_LM,POS_LW,POS_RB,POS_RM,POS_RW,POS_SS,Club Value,Country Value,League Value,Position Value
0,Miroslav Klose,184.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
1,Miroslav Klose,184.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
2,Miroslav Klose,184.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
3,Miroslav Klose,184.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
4,Miroslav Klose,184.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879


In [15]:
df.to_csv("EncodedData.csv", index=False)
df.head()

,Name,Height,Year,Club Goals,Club Goals Conceded,Goals,Assists,Red Cards,Yellow Cards,Minutes Played,...,POS_LM,POS_LW,POS_RB,POS_RM,POS_RW,POS_SS,Club Value,Country Value,League Value,Position Value
0,Miroslav Klose,184.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
1,Miroslav Klose,184.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
2,Miroslav Klose,184.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
3,Miroslav Klose,184.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879
4,Miroslav Klose,184.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,...,0,0,0,0,0,0,6.894101,6.85682,6.85629,6.726879


In [19]:
df.shape

(120799, 33)